# Transfer learning for image classification

(Original: Rodrigo Agundez)

In [1]:
import os

os.environ['KERAS_BACKEND'] = 'tensorflow'

In [2]:
import os
import random

from pprint import pprint

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np

from keras import datasets
from keras import layers
from keras.models import Model
from keras.preprocessing import image

# from utils import plot_training_summary
# from utils import TimeSummary
# from utils import save_keras_dataset_to_disk

%matplotlib inline

Using TensorFlow backend.


In [3]:
plt.rcParams['figure.figsize'] = 15, 6

## Pre-trained models available in Keras

There are several pre-trained models for image classification in Keras. Here is a summary:

![pre-trained models](images/pre_trained_models.png)

> ### extras
> 
Have a look at the documentation about the different pre-trained models https://keras.io/applications/

## Inception V3

Trained for `imagenet` over 1000 classes. Check the classes it was trained on [here](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).

![inception V3](images/inceptionV3.png)

## MobileNet

Trained for `imagenet` over 1000 classes. Check the classes it was trained on [here](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).

![MobileNet](images/mobilenet.jpeg)

### Predict image using [`MobileNet`](https://keras.io/applications/#mobilenet)

The model has been pre-trained on [ImageNet 1000 classes](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).

In [4]:
from keras.applications.mobilenet import decode_predictions
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input

In [ ]:
model = MobileNet(weights='imagenet') # this can take a bit
model.summary()

 2146304/17225924 [==>...........................] - ETA: 4:40

try the default first and then add your own image to the `image` directory

In [ ]:
img_path = 'images/scorpion.jpg'
plt.imshow(mpimg.imread(img_path));

By default it expects an image of input shape of `(224, 224, 3)`

In [ ]:
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = decode_predictions(model.predict(x), top=5)[0]
preds = [(x[1], x[2]) for x in preds]
pprint(preds)

In [ ]:
# del model

## Data 

We will use the CIFAR10 dataset.

In [ ]:
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()
[ax.imshow(random.choice(X_train), cmap='gray') for ax in plt.subplots(1, 6)[1]];

to simulate a more realistic scenario I'll save the images to disk and we will ingest them from there to the model input

In [ ]:
# takes around 1.5 minutes and occupies ~50MB
save_keras_dataset_to_disk(X_train, y_train, X_test, y_test)

In [ ]:
# this only works if you have the tree command
! tree -L 2 data/CIFAR10

## Use pre-trained `MobileNet` model as base model

load model without last classification layer and set input_shape as closer as possible to original data. Not all input shapes work, check the [documentation](https://keras.io/applications/#mobilenet).

In [ ]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

compare last layers with the full model

In [ ]:
base_model.summary()

freeze weights for these layers

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

### Exercise 1: add custom layers for our CIFAR10 dataset problem (solution in `utils.py`)

using the [`functional` API](https://keras.io/getting-started/functional-api-guide/) you will:

- get the output from the base model
- add `GlobalAveragePooling2D` layer
- add `Dense` layer of 512 units
- add output `Dense` layer with 10 units
- put layers together into custom model

In [ ]:
def make_inceptionV3_custom_model(base_model):
    # get base model output
    x = base_model.output
    
    # add GlobalAveragePooling2D layer

    # add Dense layer of 512 units

    # add output Dense layer with 10 units and softmax activation function
#     predictions =
    
    # put layers together into custom model
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

model = make_inceptionV3_custom_model(base_model)
model.summary()

> **extra**
>
> Check how many trainable parameters in the summary, does it make sense?

trainable layers

In [ ]:
for l in model.layers:
    if l.trainable:
        print(l)

In [ ]:
model = make_inceptionV3_custom_model(base_model)
model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## Exercise 2: Ingest `CIFAR10` data from file

Implement [`ImageDataGenerator`](https://keras.io/preprocessing/image/) such that:
- pixel values are re-scaled in the interval 0 to 1
- when changing the image size to (128X128) the missing pixels are filled with 0 

In [ ]:
train_datagen = image.ImageDataGenerator(
)

Implement [`ImageDataGenerator.flow_from_directory`](https://keras.io/preprocessing/image/) such that:
- we read the images from `data/CIFAR10/train`
- the target size is set to 128x128
- the class vairable is set to categorical values (10 classes)
- ensure shuffle
- and the betch size is set by us

In [ ]:
batch_size = 50

train_generator = train_datagen.flow_from_directory(
)

Use  [`Sequential.fit_generator`](https://keras.io/models/sequential/) such that:
- we ingest images using the `train_generator`
- train for 5 `epochs`
- use 10 `steps_per_epoch`
- use a suitable `max_queue_size` (check your memory consumption)

In [ ]:
time_summary = TimeSummary()
summary = model.fit_generator(
    # fill here
    verbose=1,
    callbacks=[time_summary]
)

In [ ]:
plot_training_summary(summary, time_summary)

> **extra**
>
> Make a validator data generator and pass it to the [`Sequential.fit_generator`](https://keras.io/models/sequential/)